In [1]:
import pandas
import xml.etree.ElementTree as etree
import numpy
import sqlite3

## Lecture des données 

In [2]:
#CSV File 
df1 = pandas.read_csv("TP1_donnees/adult1.csv",skipinitialspace=True)
df1.head()

,age,workclass,education,Marital-status,occupation,sex,Hours-per-week,class
0,39.0,State-gov,Bachelors,Never-married,Adm-clerical,Male,40,<=50K
1,50.0,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Male,13,<=50K
2,38.0,Private,HS-grad,Divorced,Handlers-cleaners,Male,40,<=50K
3,53.0,Private,11th,NaN,Handlers-cleaners,Male,40,<=50K
4,28.0,Private,Bachelors,Married-civ-spouse,Prof-specialty,Female,40,<=50K


In [3]:
#CSV file without columns names and ; as separator
noms = ["class","age","sex","workclass","education","hours-per-week","marital-status"]
df2 = pandas.read_csv("TP1_donnees/adult2.csv",skipinitialspace=True,sep=";",header=None,names=noms)
df2.head()

,class,age,sex,workclass,education,hours-per-week,marital-status
0,N,25,F,Private,Some-college,40,Married-civ-spouse
1,N,18,F,Private,HS-grad,30,Never-married
2,Y,47,F,"Private, Prof-school",60,Married-civ-spouse,NaN
3,Y,50,M,Federal-gov,Bachelors,55,Divorced
4,N,47,M,Self-emp-inc,HS-grad,60,Divorced


In [4]:
#SQLlite file
con = sqlite3.connect("TP1_donnees/adult3.db")
df3 = pandas.read_sql_query("SELECT * FROM income", con)
df3 = df3.replace('?', numpy.nan)
df3.head()

,num,age,workclass,education,marital-status,sex,hours-per-day,class
0,1,76,Private,Masters,married,M,8.0,Y
1,2,44,Private,Bachelors,married,M,12.0,Y
2,3,47,Self-emp-not-inc,Masters,single,F,10.0,N
3,4,20,Private,Some-college,single,F,8.0,N
4,5,29,Private,HS-grad,single,M,8.0,N


In [5]:
#XML File
tree = etree.parse("TP1_donnees/adult4.xml")
root = tree.getroot()

noms2 = ["id", "age", "workclass", "education", "marital-status", "sex", "hours-per-week", "class"]
df4 = pandas.DataFrame(columns=noms2)

def valeur_noeud(noeud):
    return noeud.text if noeud is not None else numpy.nan

for candidat in root: 

    idi = candidat.get("id")
    age = valeur_noeud(candidat.find("age"))
    workclass = valeur_noeud(candidat.find("workclass"))
    education = valeur_noeud(candidat.find("education"))
    marital = valeur_noeud(candidat.find("marital-status"))
    sex = valeur_noeud(candidat.find("sex"))
    hours = valeur_noeud(candidat.find("hours-per-week"))
    klass = valeur_noeud(candidat.find("class"))
    df4 = df4.append(pandas.Series([idi, age, workclass, education, marital, sex, hours, klass],index=noms2), ignore_index=True)

df4.head()

,id,age,workclass,education,marital-status,sex,hours-per-week,class
0,52,47,Local-gov,Some-college,divorced,F,38,N
1,53,34,Private,HS-grad,single,F,40,N
2,54,33,Private,Bachelors,single,F,40,N
3,55,21,Private,HS-grad,single,M,35,N
4,56,52,NaN,HS-grad,divorced,M,45,Y


## Intégration des données

In [6]:
df3.rename(columns={"num": "id", "hours-per-day": "hours-per-week"}, inplace=True)
df1.rename(columns={"Hours-per-week": "hours-per-week", "Marital-status": "marital-status"}, inplace=True)

In [7]:
ordre = ["age", "workclass", "education", "marital-status", "sex", "hours-per-week", "class"]
#On met occupation en dernier vu qu'elle éxiste seuelement dans le premier df
df1 = df1.reindex(ordre + ["occupation"], axis=1)
df2 = df2.reindex(ordre, axis=1)
df3 = df3.reindex(ordre + ["id"], axis=1)
df4 = df4.reindex(ordre + ["id"], axis=1)

In [8]:
#Dans le fichier original c'était hours per day
df3["hours-per-week"] *= 5

In [9]:
# concaténer les enregistrements des deux tables
df34 = pandas.concat([df3, df4], ignore_index=True)
# définir le type de "id" comme étant entier, et remplacer la colonne
df34["id"] = pandas.to_numeric(df34["id"], downcast="integer")
# ordonner les enregistrements par "id"
df34 = df34.sort_values(by="id")
# regrouper les par "id", et pour chaque groupe remplacer les
# valeurs absentes par une valeur précédente dans le même groupe
#print(df34)
#df34 = df34.groupby("id").ffill()
# supprimer les enregistrements dupliqués
# on garde les derniers, puisqu'ils sont été réglés
df34.drop_duplicates("id", keep="last", inplace=True)

In [10]:
df1.drop(["occupation"], axis=1, inplace=True)
df34.drop(["id"], axis=1, inplace=True)

In [11]:
dic = {
    "Never-married": "single",
    "Married-civ-spouse": "married",
    "Married-spouse-absent": "married",
    "Married-AF-spouse": "married",
    "Divorced": "divorced",
    "Separated": "divorced",
    "Widowed": "widowed"
}
df1["marital-status"] = df1["marital-status"].map(dic)
df2["marital-status"] = df2["marital-status"].map(dic)

In [12]:
df1["sex"] = df1["sex"].map({"Female": "F", "Male": "M"})

In [13]:
df1["class"] = df1["class"].map({"<=50K": "N", ">50K": "Y"})

In [14]:
df = pandas.concat([df1, df2, df34], ignore_index=True)

In [15]:
df

,age,workclass,education,marital-status,sex,hours-per-week,class
0,39,State-gov,Bachelors,single,M,40,N
1,50,Self-emp-not-inc,Bachelors,married,M,13,N
2,38,Private,HS-grad,divorced,M,40,N
3,53,Private,11th,NaN,M,40,N
4,28,Private,Bachelors,married,F,40,N
...,...,...,...,...,...,...,...
189,58,State-gov,Doctorate,married,M,1,Y
190,36,Private,Some-college,divorced,M,40,N
191,40,Federal-gov,Masters,single,F,55,Y
192,35,Private,Masters,married,M,40,Y


In [16]:
print(df.isnull().sum())

age                6
workclass         10
education          1
marital-status     4
sex                3
hours-per-week     2
class              0
dtype: int64


In [18]:
df.dropna(subset=["workclass", "education", "marital-status", "sex", "hours-per-week", "class"], inplace=True)

In [21]:
df["age"] = pandas.to_numeric(df["age"])
df["age"] = df.groupby(["class", "education"])["age"].transform(lambda x: x.fillna(int(round(x.mean()))))

In [25]:
df["hours-per-week"] = pandas.to_numeric(df["hours-per-week"])

In [26]:
df.head()

,age,workclass,education,marital-status,sex,hours-per-week,class
0,39.0,State-gov,Bachelors,single,M,40.0,N
1,50.0,Self-emp-not-inc,Bachelors,married,M,13.0,N
2,38.0,Private,HS-grad,divorced,M,40.0,N
4,28.0,Private,Bachelors,married,F,40.0,N
5,37.0,Private,Masters,married,F,40.0,N
